## Readind data and convert to ndarray

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.random.seed(10)

In [3]:
df_train = pd.read_csv('F:/mnist/data/train.csv')

In [4]:
tmp_df = df_train
print(len(tmp_df))
tmp_df.head()

42000


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
x_train = np.array(tmp_df.ix[:,1:785])
y_train = np.array(tmp_df.ix[:,0])

In [6]:
import keras

Using TensorFlow backend.


In [7]:
y_labels = keras.utils.to_categorical(y_train, num_classes=10)

## Play with given data set (mnist)

In [8]:
from keras.datasets import mnist

In [9]:
# load data
# (X_train, y_train), (X_test, y_test) = mnist.load_data()

## Call the sequential model

In [10]:
from keras.models import Sequential

from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [11]:
# call cnn layers
from keras.layers import Dense, Activation

In [12]:
num_classes = 10

In [13]:
X_train = x_train.reshape(x_train.shape[0], 1, 28, 28).astype('float32')
X_train = X_train / 255

In [14]:
def init_model():
    model = Sequential()
    model.add(Dense(units=num_classes, input_dim=784))
    model.add(Activation('relu'))
    model.add(Dense(units=10))

    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

In [15]:
def baseline_model():
# create model
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [16]:
# define the larger model
def larger_model():
# create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [17]:
model = larger_model()

In [18]:
# learn dl model
model.fit(X_train, y_labels, epochs=100, batch_size=1000, validation_split=0.2)

Train on 33600 samples, validate on 8400 samples
Epoch 1/100
33600/33600 [==============================] - 129s - loss: 1.5190 - acc: 0.5370 - val_loss: 0.4537 - val_acc: 0.8667
Epoch 2/100
33600/33600 [==============================] - 131s - loss: 0.3782 - acc: 0.8832 - val_loss: 0.2080 - val_acc: 0.9393
Epoch 3/100
33600/33600 [==============================] - 131s - loss: 0.2192 - acc: 0.9334 - val_loss: 0.1382 - val_acc: 0.9606
Epoch 4/100
33600/33600 [==============================] - 132s - loss: 0.1617 - acc: 0.9512 - val_loss: 0.1044 - val_acc: 0.9690
Epoch 5/100
33600/33600 [==============================] - 132s - loss: 0.1300 - acc: 0.9613 - val_loss: 0.0860 - val_acc: 0.9733
Epoch 6/100
33600/33600 [==============================] - 132s - loss: 0.1094 - acc: 0.9675 - val_loss: 0.0756 - val_acc: 0.9755
Epoch 7/100
33600/33600 [==============================] - 132s - loss: 0.0946 - acc: 0.9712 - val_loss: 0.0684 - val_acc: 0.9776
Epoch 8/100
33600/33600 [================

## Prediction test data

In [19]:
df_test = pd.read_csv('F:/mnist/data/test.csv')

In [20]:
x_test = np.array(df_test)

In [21]:
X_test = x_test.reshape(x_test.shape[0], 1, 28, 28).astype('float32')
X_test = X_test / 255

In [22]:
# this method will take model and numpy array data
def get_submission_df(model, data):
    np_prediction = model.predict_classes(data)
    
    df_submission = pd.DataFrame()
    df_submission['ImageId'] = list(range(1,len(np_prediction)+1))
    df_submission['Label'] = np_prediction
    
    return df_submission

In [23]:
df_submission = get_submission_df(model, X_test)

28000/28000 [==============================] - 62s    


In [24]:
df_submission.to_csv('F:/mnist/data/submission.csv', index=False)